In [2]:
!pip install neo4j-driver
from neo4j import GraphDatabase, basic_auth
import pandas as pd

     |████████████████████████████████| 89 kB 3.3 MB/s 
  Created wheel for neo4j-driver: filename=neo4j_driver-4.4.0-py3-none-any.whl size=114976 sha256=1bd7e397d7825c31592252ddea3523821808147040285b904f6a2d432e8ee297
  Stored in directory: /root/.cache/pip/wheels/5e/89/23/48d22f30bfad0866aac1078fb2e24db1dd3750c46198557ffe
Successfully built neo4j-driver


In [3]:
driver = GraphDatabase.driver(
  "bolt://3.235.155.226:7687",
  auth=basic_auth("neo4j", "spring-acronyms-assembly"))

In [4]:
def read_cypher(query):
    with driver.session(database="neo4j") as session:
        results = session.run(query).data()
        df = pd.DataFrame(results)
        col_names = df.columns.values.tolist()
        for c in col_names:
            if type(df[c].iloc[0]) is dict:
                df = pd.concat([df, df[c].apply(pd.Series)], axis=1)
                df.drop(columns=c, inplace=True)
            else:
                df1 = df.pop(c)
                df[c] = df1
        return df

In [ ]:
#1. Il cognome e il nome dei poliziotti presenti nel database. Ordinare in ordine alfabetico.

cypher_query = '''

MATCH (o:Officer)
RETURN o.name,o.surname ORDER BY o.name

'''

read_cypher(cypher_query)

,o.name,o.surname
0,Abagael,McArthur
1,Abbi,Gove
2,Abel,Hierro
3,Abraham,Henbury
4,Adair,Glasby
...,...,...
995,Yul,Buddle
996,Zackariah,Rosenfrucht
997,Zeb,Blazewski
998,Zelda,Pattinson


In [ ]:
#2. Il cognome e il nome dei poliziotti che hanno il grado (rank) di &#39;Police Constable&#39;

cypher_query = '''

MATCH (o:Officer {rank:'Police Constable'})
RETURN o.name,o.surname

'''

read_cypher(cypher_query)

,o.name,o.surname
0,Christian,Hannam
1,Aguie,Clampett
2,Von,Death
3,Isak,Basley
4,Marketa,Aykroyd
...,...,...
370,Eloisa,Mayoh
371,Hortense,Shearme
372,Kassey,Nias
373,Hertha,Emerine


In [ ]:
#3. I crimini in cui sono coinvolte BMW modello M o Mercedes-Benz modello M-Class (model)

cypher_query = '''

MATCH (v:Vehicle)-[:INVOLVED_IN]->(c:Crime)
WHERE v.make = 'BMW' AND v.model = 'M' OR v.make = 'Mercedes-Benz' AND v.model = 'M-Class'
RETURN c

'''

read_cypher(cypher_query)

,date,id,type,last_outcome
0,4/08/2017,a42699bed37958abb8c9096df66a3d533b17540b632141...,Vehicle crime,Investigation complete; no suspect identified
1,28/08/2017,da187294ebd822e6651308e4a2987bbf56e0ab2c9d22d9...,Vehicle crime,Investigation complete; no suspect identified
2,4/08/2017,547b8f2b12aea51b8f0eb229aaa4ec0370f4e67c7d3e1d...,Vehicle crime,Investigation complete; no suspect identified
3,29/08/2017,74dfbc7365739859002b3f3f8e932975e3c64350a564c3...,Vehicle crime,Investigation complete; no suspect identified


In [ ]:
#4. I crimini seguiti dal poliziotto che si chiama Christian Hannam

#In questo caso se esistessero due Hannam allora bisognerebbe specificare anche il rank.

cypher_query = '''

MATCH (c:Crime)-[:INVESTIGATED_BY]->(o:Officer)
WHERE o.name = 'Christian' AND o.surname = 'Hannam'
RETURN c

'''

read_cypher(cypher_query)

,date,id,last_outcome,type
0,11/08/2017,23f4a74d85130c92c496a17bb3f6d80dac6cfa8773e76c...,Unable to prosecute suspect,Violence and sexual offences
1,19/08/2017,a1284e44195941819799a20f634dd5bc0c6a599ef1d9e0...,Local resolution,Criminal damage and arson
2,4/08/2017,aacff8850d0af4537ec79fb1c8869a560b9925a73441df...,Investigation complete; no suspect identified,Vehicle crime
3,26/08/2017,6d51d17dd4fdb62d5d108ebefc6c3cc8894f50f5a17c92...,Offender given a drugs possession warning,Drugs
4,11/08/2017,f4713af2c4e19ecf05885b405104d286c3cd1df696814c...,Further investigation is not in the public int...,Criminal damage and arson
5,28/08/2017,cf41ea73b3aafa8097360cedf178603d4246309050e75f...,Investigation complete; no suspect identified,Vehicle crime
6,21/08/2017,665beb93dbf15a1870392a6d5484bf956f46f73e4fdd67...,Investigation complete; no suspect identified,Bicycle theft
7,15/08/2017,19a2059b8e2a0b509412b80e732c2b762b7ebcfc5f24db...,Unable to prosecute suspect,Violence and sexual offences
8,7/08/2017,3ee432b421e9baf4873f7726b4d2acab75bf7f79b0e215...,Investigation complete; no suspect identified,Theft from the person
9,12/08/2017,0fcb44efce1647a3681bb74f9c7ff7c5484d458e7eee0f...,Investigation complete; no suspect identified,Other theft


In [5]:
#5. Il tipo dei crimini seguiti da Christian Hannam che sono ancora 'under investigation'

cypher_query = '''

MATCH (c:Crime {last_outcome:"Under investigation"})-[:INVESTIGATED_BY]->(o:Officer)
WHERE o.name = 'Christian' AND o.surname = 'Hannam'
RETURN c

'''

read_cypher(cypher_query)


,date,id,last_outcome,type
0,10/08/2017,e6537cce7e5e701176646e29d4044253189c4cfb51d5ca...,Under investigation,Criminal damage and arson
1,18/08/2017,3ff357e11bda6193e32714678b32cb0fc79df88d493c7e...,Under investigation,Robbery


In [ ]:
#6. Gli indirizzi dei crimini su cui Christian Hannam sta ancora investigando

cypher_query = '''

MATCH (o:Officer)<-[:INVESTIGATED_BY]-(c:Crime{last_outcome:"Under investigation"})-[:OCCURRED_AT]->(l:Location)
WHERE o.name = 'Christian' AND o.surname = 'Hannam'
RETURN l.address

'''

read_cypher(cypher_query)

,l.address
0,13 Stovell Road
1,114 Wigan Road


In [ ]:
#7. L’area dei crimini su cui Christian Hannam sta ancora investigando

cypher_query = '''

MATCH (o:Officer)<-[:INVESTIGATED_BY]-(c:Crime{last_outcome:"Under investigation"})-[:OCCURRED_AT]->(l:Location)-[:LOCATION_IN_AREA]->(a:Area)
WHERE o.name = 'Christian' AND o.surname = 'Hannam'
RETURN a.areaCode

'''

read_cypher(cypher_query)

,a.areaCode
0,M40
1,BL3


In [ ]:
#8. Il numero di poliziotti per ogni grado. Ordinare dal grado che ha più poliziotti a scendere.

cypher_query = '''

MATCH (o:Officer)
RETURN o.rank, count(o.rank) ORDER BY count(o.rank) DESC

'''

read_cypher(cypher_query)

,o.rank,count(o.rank)
0,Police Constable,375
1,Sergeant,336
2,Inspector,199
3,Chief Inspector,90


In [ ]:
#9. Il grado che ha meno di 200 poliziotti

#nell'espressione WITH bisogna assolutamente utilizzare "AS"

cypher_query = '''

MATCH (o:Officer)
WITH o.rank AS rank, COUNT(o.rank) AS tot
WHERE tot < 200
RETURN rank,tot

'''

read_cypher(cypher_query)

,rank,tot
0,Inspector,199
1,Chief Inspector,90
